In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
from model import load_config, construct_sensor_data_class,setup_database, get_session,setup_database_engine, create_config_file, load_data_to_db
import yaml
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime
from sqlalchemy.orm import sessionmaker

Base = declarative_base()

In [2]:
database_uri="sqlite:///moh_iot_data.db"

In [3]:



def preprocess_sensor_data(df):
    # drop empty rows/cols
    df = df.iloc[2:, 4:].reset_index(drop=True)
    # Set the column names to the values in the first row
    df.columns = df.iloc[0]
    # Drop the first row as it's now the header
    df = df.drop(df.index[0])
    df = df.reset_index(drop=True)
    # rename cols
    df.columns = [col.replace(' - SnapShot', '') for col in df.columns]
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    df.columns = [col.replace('.pv', '') for col in df.columns]
    try:
        df = df.set_index('timestamp')
        df.index = pd.to_datetime(df.index)
    except Exception as e:
        print(f'Exception at preprocess_sensor_data():\n{e}')
        pass
    for col in df.columns:
        # Convert only if the column type is 'object'
        if df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

def rename_numeric_columns(df):
    # Select columns where the data type is numeric
    numeric_cols = df.select_dtypes(include=['number']).columns

    # Create a dictionary for renaming columns that start with a number
    rename_dict = {col: f'sensor_{col}' for col in numeric_cols if col[0].isdigit()}

    # Rename the selected columns
    df.rename(columns=rename_dict, inplace=True)
    return df

In [4]:
uploaded_file = '~/Jupyter Notebooks/FAME/data/KT2201_v10.xlsx'
readings = pd.read_excel(uploaded_file, sheet_name='Parameters')

In [5]:
readings = preprocess_sensor_data(readings)

In [6]:
readings = rename_numeric_columns(readings)

In [7]:
def construct_sensor_data_class(config):
    attrs = {'__tablename__': config['sensor_data']['table_name'], '__table_args__': {'extend_existing': True}}
    print("Configuring columns for table:", config['sensor_data']['table_name'])
    for column in config['sensor_data']['columns']:
        column_type = column['type']
        column_name = column['name']
        kwargs = {'primary_key': column.get('primary_key', False)}  # Handle primary_key

        # Prepend 'sensor_' to column names that start with a number
        if column_name[0].isdigit():
            column_name = 'sensor_' + column_name
        print(f"Configuring column: {column_name} of type {column_type}")

        # Dynamically assign the column type based on the configuration
        if column_type == 'Integer':
            attrs[column_name] = Column(Integer, **kwargs)
        elif column_type == 'String':
            attrs[column_name] = Column(String, **kwargs)
        elif column_type == 'Float':
            attrs[column_name] = Column(Float, **kwargs)
        elif column_type == 'DateTime':
            attrs[column_name] = Column(DateTime, **kwargs)
        else:
            raise ValueError(f"Unsupported column type {column_type}")

    # Create a new class type with all attributes
    return type('SensorData', (Base,), attrs)


In [8]:
create_config_file(readings, 'KT2201', database_uri=database_uri, file_path='config-sensor.yaml' )

In [9]:
SensorData = construct_sensor_data_class(load_config()) 


Configuring columns for table: KT2201
Configuring column: timestamp of type DateTime
Configuring column: sensor_22si101 of type Float
Configuring column: sensor_22vi01 of type Float
Configuring column: sensor_22vi04 of type Float
Configuring column: sensor_22vi06 of type Float
Configuring column: sensor_22vi08 of type Float
Configuring column: sensor_22pi69 of type Float
Configuring column: sensor_22pi70 of type Float
Configuring column: sensor_22zi10 of type Float
Configuring column: sensor_22zi09 of type Float
Configuring column: sensor_22zi11 of type Float


In [10]:
config = load_config()
engine = setup_database(config)
session = get_session(engine)

In [11]:
config

{'database_uri': 'sqlite:///moh_iot_data.db',
 'sensor_data': {'columns': [{'name': 'timestamp',
    'primary_key': True,
    'type': 'DateTime'},
   {'name': 'sensor_22si101', 'type': 'Float'},
   {'name': 'sensor_22vi01', 'type': 'Float'},
   {'name': 'sensor_22vi04', 'type': 'Float'},
   {'name': 'sensor_22vi06', 'type': 'Float'},
   {'name': 'sensor_22vi08', 'type': 'Float'},
   {'name': 'sensor_22pi69', 'type': 'Float'},
   {'name': 'sensor_22pi70', 'type': 'Float'},
   {'name': 'sensor_22zi10', 'type': 'Float'},
   {'name': 'sensor_22zi09', 'type': 'Float'},
   {'name': 'sensor_22zi11', 'type': 'Float'}],
  'table_name': 'KT2201'}}

In [13]:
for column in SensorData.__table__.columns:
            print(column.name)

timestamp
sensor_22si101
sensor_22vi01
sensor_22vi04
sensor_22vi06
sensor_22vi08
sensor_22pi69
sensor_22pi70
sensor_22zi10
sensor_22zi09
sensor_22zi11


In [16]:
def load_data_to_db(df, session, SensorData):
    for _, row in df.iterrows():
        # Prepare the data for the database insertion
        data_dict = {}
        for column in SensorData.__table__.columns:
            column_name = column.name
            if column_name == 'timestamp':
                # Convert timestamp to datetime if it's not already
                try:
                    data_dict[column_name] = pd.to_datetime(row[column_name]) if not isinstance(row[column_name],
                                                                                                datetime) else row[
                        column_name]
                except KeyError:
                    # Fallback if 'timestamp' is expected to be the DataFrame index
                    data_dict[column_name] = pd.to_datetime(_) if not isinstance(_, datetime) else _
            else:
                try:
                    data_dict[column_name] = row.get(column_name)
                except Error as e:
                    print(e)
        # Create an instance of SensorData using the prepared dictionary
        sensor_data = SensorData(**data_dict)
        session.add(sensor_data)
        

    try:
        session.commit()
    except Exception as e:
        session.rollback()  # Roll back in case of error during commit
        raise e

In [133]:
sensor_data = SensorData(**d)

In [136]:
import sqlite3
import pandas as pd

# Create a connection object using the connect function
connection = sqlite3.connect('your_database_file.db')

# Write your SQL query
query = "SELECT * FROM your_table_name"

# Use read_sql_query to read the query into a DataFrame
df = pd.read_data(query, connection)

# Don't forget to close the connection when you're done
connection.close()

# Now you can work with the DataFrame 'df' as needed
print(df)


AttributeError: 'SensorData' object has no attribute 'attrs'

In [17]:
readings.iloc[:50]

,sensor_22si101,sensor_22vi01,sensor_22vi04,sensor_22vi06,sensor_22vi08,sensor_22pi69,sensor_22pi70,sensor_22zi10,sensor_22zi09,sensor_22zi11
timestamp,,,,,,,,,,
2023-08-15 00:00:00,9165.967773,10.606638,10.530332,17.092712,24.647081,9.144601,8.412057,0.165204,0.395651,0.333842
2023-08-15 00:00:10,9160.625977,10.530332,10.530332,17.092712,24.647081,9.156811,8.418161,0.163678,0.394124,0.332316
2023-08-15 00:00:20,9165.967773,10.606638,10.606638,17.397940,24.799694,9.150706,8.418161,0.164441,0.394887,0.333079
2023-08-15 00:00:30,9155.284180,10.454025,10.530332,18.237314,25.486454,9.126287,8.399847,0.164441,0.394124,0.333079
2023-08-15 00:00:40,9149.942383,10.606638,10.606638,17.321632,24.494467,9.144601,8.405951,0.165204,0.396414,0.334605
2023-08-15 00:00:50,9149.942383,10.606638,10.530332,17.626860,25.104921,9.126287,8.399847,0.165967,0.396414,0.334605
2023-08-15 00:01:00,9155.284180,10.606638,10.606638,17.855780,25.028616,9.120183,8.387638,0.165204,0.396414,0.334605
2023-08-15 00:01:10,9144.600586,10.530332,10.606638,17.016407,24.418159,9.150706,8.412057,0.165204,0.395651,0.333842
2023-08-15 00:01:20,9144.600586,10.530332,10.454025,17.245327,24.952309,9.132392,8.399847,0.163678,0.394887,0.333079


In [23]:

load_data_to_db(readings.iloc[60:100], session, SensorData)


In [25]:
def check_data_in_db(session, SensorData):
    try:
        # Query the first ten records from the SensorData table
        records = session.query(SensorData).limit(50).all()
        if records:
            for record in records:
                # Assuming you have fields like 'sensor_id', 'timestamp', 'value' in your SensorData
                # Adjust these fields based on your actual SensorData definition
                print(f"Sensor ID: {record.sensor_22pi70}, Timestamp: {record.timestamp}")

        else:
            print("No records found in the database.")
    except SQLAlchemyError as e:
        print(f"An error occurred: {e}")


# Assuming your previous setup
config = load_config()  # Load your configuration settings
engine = setup_database(config)
session = get_session(engine)

# Now, use the function to check for data
check_data_in_db(session, SensorData)

Sensor ID: 8.50362491607666, Timestamp: 2023-08-15 00:08:20
Sensor ID: 8.485310554504395, Timestamp: 2023-08-15 00:08:30
Sensor ID: 8.485310554504395, Timestamp: 2023-08-15 00:08:40
Sensor ID: 8.479206085205078, Timestamp: 2023-08-15 00:08:50
Sensor ID: 8.473101615905762, Timestamp: 2023-08-15 00:09:00
Sensor ID: 8.479206085205078, Timestamp: 2023-08-15 00:09:10
Sensor ID: 8.479206085205078, Timestamp: 2023-08-15 00:09:20
Sensor ID: 8.479206085205078, Timestamp: 2023-08-15 00:09:30
Sensor ID: 8.466997146606445, Timestamp: 2023-08-15 00:09:40
Sensor ID: 8.466997146606445, Timestamp: 2023-08-15 00:09:50
Sensor ID: 8.454788208007812, Timestamp: 2023-08-15 00:10:00
Sensor ID: 8.454788208007812, Timestamp: 2023-08-15 00:10:10
Sensor ID: 8.454788208007812, Timestamp: 2023-08-15 00:10:20
Sensor ID: 8.448683738708496, Timestamp: 2023-08-15 00:10:30
Sensor ID: 8.448683738708496, Timestamp: 2023-08-15 00:10:40
Sensor ID: 8.448683738708496, Timestamp: 2023-08-15 00:10:50
Sensor ID: 8.442579269409

In [20]:
config

{'database_uri': 'sqlite:///moh_iot_data.db',
 'sensor_data': {'columns': [{'name': 'timestamp',
    'primary_key': True,
    'type': 'DateTime'},
   {'name': 'sensor_22si101', 'type': 'Float'},
   {'name': 'sensor_22vi01', 'type': 'Float'},
   {'name': 'sensor_22vi04', 'type': 'Float'},
   {'name': 'sensor_22vi06', 'type': 'Float'},
   {'name': 'sensor_22vi08', 'type': 'Float'},
   {'name': 'sensor_22pi69', 'type': 'Float'},
   {'name': 'sensor_22pi70', 'type': 'Float'},
   {'name': 'sensor_22zi10', 'type': 'Float'},
   {'name': 'sensor_22zi09', 'type': 'Float'},
   {'name': 'sensor_22zi11', 'type': 'Float'}],
  'table_name': 'KT2201'}}

In [21]:
import sqlite3
conn = sqlite3.connect('moh_iot_data.db')


In [26]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM KT2201 limit 10')
rows = cursor.fetchall()
for row in rows:
    print(row)


('2023-08-15 00:08:20.000000', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 9171.30859375, 10.682945251464844, 10.682945251464844, 17.397939682006836, 25.25753402709961, 9.242274284362793, 8.50362491607666, 0.16520416736602783, 0.39946579933166504, 0.3376573324203491)
('2023-08-15 00:08:30.000000', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 9149.9423828125, 10.911865234375, 10.682945251464844, 17.09271240234375, 24.723388671875, 9.223959922790527, 8.485310554504395, 0.16596722602844238, 0.40022897720336914, 0.3391835689544678)
('2023-08-15 00:08:40.000000', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 9155.2841796875, 10.835558891296387, 10.606637954711914, 18.237314224243164, 25.867990493774414, 9.223959922790527, 8.485310554504395, 0.16749334335327148, 0.4009920358657837, 0.3391835689544678)
('2023-08-15 00:08:50.000000', None, None, None, None, Non

In [27]:
row

('2023-08-15 00:09:50.000000',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 9155.2841796875,
 10.911865234375,
 10.682945251464844,
 17.24532699584961,
 24.723388671875,
 9.205646514892578,
 8.466997146606445,
 0.16596722602844238,
 0.39946579933166504,
 0.3376573324203491)

In [106]:
conn.close()

In [69]:
readings.iloc[49]

sensor_22si101    9165.967773
sensor_22vi01       10.835559
sensor_22vi04       10.682945
sensor_22vi06       17.245327
sensor_22vi08       24.799694
sensor_22pi69        9.242274
sensor_22pi70        8.503625
sensor_22zi10        0.165967
sensor_22zi09        0.400992
sensor_22zi11        0.338420
Name: 2023-08-15 00:08:10, dtype: float64

In [93]:
row

('2023-08-15 00:09:50.000000',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 9155.2841796875,
 10.911865234375,
 10.682945251464844,
 17.24532699584961,
 24.723388671875,
 9.205646514892578,
 8.466997146606445,
 0.16596722602844238,
 0.39946579933166504,
 0.3376573324203491)